In [11]:
import pandas as pd
import zipfile
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/repl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/repl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/repl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Pipeline de Processamento :

- **Entrada do Usuário:** A mensagem do usuário é capturada e pré-processada (remoção de ruídos, tokenização).
- **Word2Vec:** Gera embeddings das palavras para capturar o significado semântico.
- **Análise de Sentimentos:** Avalia o tom emocional da mensagem.
- **RNN:** Processa os embeddings e o histórico da conversa para gerar uma resposta contextualizada.
- **Saída:** O chatbot gera uma resposta personalizada com base nas intenções detectadas, no contexto e no sentimento do usuário.

In [3]:
zip_file_path = 'sentiment+labelled+sentences.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall()

# 
amazon = pd.read_csv("sentiment labelled sentences/amazon_cells_labelled.txt", delimiter="\t", header=None, names=["sentence", "sentiment"])
imdb = pd.read_csv("sentiment labelled sentences/imdb_labelled.txt", delimiter="\t", header=None, names=["sentence", "sentiment"])
yelp = pd.read_csv("sentiment labelled sentences/yelp_labelled.txt", delimiter="\t", header=None, names=["sentence", "sentiment"])

df = pd.concat([imdb, yelp, amazon], ignore_index=True)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2748 entries, 0 to 2747
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentence   2748 non-null   object
 1   sentiment  2748 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 43.1+ KB


,sentence,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


## Uso do Word2Vec
O Word2Vec transforma o texto em vetores numéricos de alta dimensão capaz de capturar as relações semânticas e sintaáticas entre as palavras. Então ela ajuda ao chatbot a identificar as diferenças variações da pergunta e contexto da situação. 

In [4]:
def preprocess(text):
    # Tokenização
    tokens = nltk.word_tokenize(text.lower())
    # Remoção de pontuação e stopwords
    tokens = [word for word in tokens if word.isalpha() and word not in stopwords.words('english')]
    # Lematização
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

In [5]:
# Pré-processamento das frases
df['processed_sentence'] = df['sentence'].apply(preprocess)

In [6]:
model_word2vec = Word2Vec(sentences=[sentence.split() for sentence in df['processed_sentence']], vector_size=100, window=5, min_count=1, workers=4)

In [7]:
def sentence_vector(sentence):
    # Calcular a média dos vetores de palavras na frase
    words = sentence.split()
    word_vectors = [model_word2vec.wv[word] for word in words if word in model_word2vec.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model_word2vec.vector_size)

import numpy as np

# Criar vetores de frases
X_vectors = np.array([sentence_vector(sentence) for sentence in df['processed_sentence']])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_vectors, df['sentiment'], test_size=0.2, random_state=42)

## Impacto das RNN no chatbot

O modelos RNNs, especialmente arquiteturas como LSTM (Long Short-Term Memory) e GRU (Gated Recurrent Unit), são capazes de processar sequências de dados e capturar dependências de longo prazo. Isso permite que o chat bot mantenha uma meméria ao longo da conversa, dê respostas mais coerentes e contextuais, além de identificar sentimentos.No cenário de atendimento ao cliente isso é importante para evitar atritos entre o cliente e a empresa, já que o atendimento irá se aproximar mais do humano. 

In [9]:
# Parâmetros do modelo
vocab_size = 1000
embedding_dim = 100
lstm_units = 128

model_rnn = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=10),
    LSTM(lstm_units),
    Dense(1, activation='sigmoid')
])

model_rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
# Treinamento do modelo RNN
model_rnn.fit(X_train, y_train, epochs=10, batch_size=4, validation_data=(X_test, y_test))

Epoch 1/10
550/550 ━━━━━━━━━━━━━━━━━━━━ 27s 46ms/step - accuracy: 0.4803 - loss: 0.6945 - val_accuracy: 0.4564 - val_loss: 0.6990
Epoch 2/10
550/550 ━━━━━━━━━━━━━━━━━━━━ 26s 46ms/step - accuracy: 0.5144 - loss: 0.6934 - val_accuracy: 0.4564 - val_loss: 0.6962
Epoch 3/10
550/550 ━━━━━━━━━━━━━━━━━━━━ 25s 46ms/step - accuracy: 0.5101 - loss: 0.6932 - val_accuracy: 0.4564 - val_loss: 0.6979
Epoch 4/10
550/550 ━━━━━━━━━━━━━━━━━━━━ 25s 46ms/step - accuracy: 0.5092 - loss: 0.6936 - val_accuracy: 0.4564 - val_loss: 0.6963
Epoch 5/10
550/550 ━━━━━━━━━━━━━━━━━━━━ 25s 46ms/step - accuracy: 0.5268 - loss: 0.6920 - val_accuracy: 0.4564 - val_loss: 0.6933
Epoch 6/10
550/550 ━━━━━━━━━━━━━━━━━━━━ 25s 46ms/step - accuracy: 0.4993 - loss: 0.6933 - val_accuracy: 0.4564 - val_loss: 0.6949
Epoch 7/10
550/550 ━━━━━━━━━━━━━━━━━━━━ 25s 46ms/step - accuracy: 0.5156 - loss: 0.6929 - val_accuracy: 0.4564 - val_loss: 0.6959
Epoch 8/10
550/550 ━━━━━━━━━━━━━━━━━━━━ 25s 46ms/step - accuracy: 0.5175 - loss: 0.6928 - 

## Exemplo de interação

In [12]:
def chatbot_response(user_input):
    processed_input = preprocess(user_input)
  
    sentiment = model_rnn.predict([user_vector])[0][0] if 'user_vector' in locals() else 0  
    
    sentiment_label = 1 if sentiment >= 0.5 else 0
    
  
    keywords = ['change password', 'reset password', 'password', 'ajuda', 'suporte']
    has_keyword = any(keyword in processed_input for keyword in keywords)
    
    # Geração de resposta baseada no sentimento e palavras-chave
    if has_keyword:
        if 'change password' in processed_input or 'reset password' in processed_input:
            response = "Claro, posso ajudar você a redefinir sua senha. Por favor, siga estas etapas..."
        elif 'password' in processed_input:
            response = "Parece que você está tendo problemas com sua senha. Como posso ajudar?"
        else:
            response = "Claro, como posso ajudar você hoje?"
    else:
        if sentiment_label == 1:
            response = "Que ótimo saber que você está feliz!"
        else:
            response = "Lamento que você esteja se sentindo assim. Como posso ajudar?"
    
    return response

In [17]:
user_input = "I wanna change password"
print(chatbot_response(user_input))
user_input = "I so angry"
print(chatbot_response(user_input))

Claro, posso ajudar você a redefinir sua senha. Por favor, siga estas etapas...
Lamento que você esteja se sentindo assim. Como posso ajudar?
